## carga das tabelas

In [60]:
import pandas as pd
from glob import glob
from functools import reduce

As tabelas vêm empilhadas em arquivos texto. Cada arquivo tem um número de linhas diferente, por isso configuramos cada um:

In [40]:
filenames = sorted(glob('./rust_data/*.asc'))
linhas_por_tabela = [137,137,137,137,128,36,60,81]

Para cada arquivo, lemos uma coluna só, mas depois quebramos em várias colunas redimensionando as séries de acordo com as `linhas_por_tabela` de cada um. Então criamos uma tabela com os ids de cada ônibus como nomes de colunas, e os dados dos odômetros que estão a partir da 12a linha. As substituições de motor são salvas usando a 6a e a 9a linhas, como especificado.

In [111]:
df_crus = (pd.read_csv(filename, header=None) for filename in filenames)
series = [base.values.reshape(-1, linhas_por_tabela[i]).T for i, base in enumerate(df_crus)]
odometer = [pd.DataFrame(s[11:], columns=s[0].astype(str)) for s in series]
replacements = [pd.DataFrame([s[5],s[8]], columns=s[0].astype(str)) for s in series]

O último passo é juntar todos os frames em um só, preenchendo com zeros as colunas que não têm linhas suficientes. Então salvamos em parquet para ficar fácil de abrir num outro notebook.

In [112]:
def joiner(r, s):
    return r.join(s)

o_df = reduce(joiner, odometer).fillna(0)
r_df = reduce(joiner, replacements).fillna(0)

In [115]:
r_df.to_parquet('./rust_data/replacements.parquet', compression='gzip')

In [116]:
o_df.to_parquet('./rust_data/odometers.parquet', compression='gzip')